# Imports

In [36]:
#!pip install pyLDAvis
#!pip install visdom
import pandas as pd
import nltk
import re
import gensim
import gensim.corpora as corpora
from gensim.models import Phrases
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from sklearn.feature_extraction.text import CountVectorizer
tqdm.pandas()

# Data

In [37]:
#from google.colab import drive
#drive.mount('/content/drive')
df_lda_kickstarter = pd.read_csv('../kickstarter_cleaned.csv')

# Functions

In [38]:
def preprocess_text(document: str, stemmer: nltk.stem.WordNetLemmatizer, en_stop: set) -> list:
    """Preprocesses a document to remove special characters/whitespace/etc

    Args:
        document (str):
        stemmer (nltk.stem.WordNetLemmatizer): Stemmer from NLTK
        en_stop (set): Set of stop words, usually from NLTK

    Returns:
        str: preprocessed document
    """

    # Remove all the special characters
    document = re.sub(r'\W', ' ', document)

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Tokenization
    tokens = nltk.word_tokenize(document)

    # POS-Tagging
    tagged_tokens = nltk.pos_tag(tokens)

    # Filter Nouns and Lemmatization
    lemmatized_nouns = []
    for token, pos in tagged_tokens:
        if pos.startswith('N'):  # Check if the token is a noun
            lemma = stemmer.lemmatize(token)
            # FIlter stop words, words that contain only numbers and short words
            if lemma not in en_stop and not lemma.isdigit() and len(lemma) > 2:
                lemmatized_nouns.append(lemma)

    return lemmatized_nouns;

In [39]:
def preprocess_text_helper(t):
    stemmer = nltk.stem.WordNetLemmatizer()
    return preprocess_text(t, stemmer, stopwords.words('english'))

In [40]:
# Create function to calculate topic distribution
def get_topic_distribution(text, dictionary, model):
    bow = dictionary.doc2bow(text)
    topic_probs = model.get_document_topics(bow)
    return topic_probs

In [ ]:
# Create a function to extract the top n descriptions corresponding to a topic
def get_top_descriptions(model, df, corpus, chosen_topic, n):

    print(model.show_topic(chosen_topic, topn=10))

    # Get the document-topic distribution
    document_topic_distribution = model.get_document_topics(corpus)

    # Ensure chosen_topic is within the valid range
    if 0 <= chosen_topic < model.num_topics:
        # Sort documents by their probability score for the chosen topic
        sorted_documents = sorted(
            enumerate(document_topic_distribution),
            key=lambda x: next((prob for topic, prob in x[1] if topic == chosen_topic), 0),
            reverse=True
        )
        top_n = 5  # Number of top documents to print
        for i, (doc_id, topic_probs) in enumerate(sorted_documents[:top_n]):
            document = corpus[doc_id]
            project_description = df.iloc[doc_id]['project_description']
            processed_description = df.iloc[doc_id]['processed_description']
            print(f"Document {i + 1} (Corpus ID {doc_id}):")
            print("Topic Probability:", next((prob for topic, prob in topic_probs if topic == chosen_topic), 0))
            words = [word for word, freq in document]
            print("Document Text:", words)
            print("Project Description:", project_description)
            print("processed Description:", processed_description)
            print("\n")
    else:
        print(f"Chosen topic index {chosen_topic} is out of range.")

# LDA

In [43]:
# Remove the columns
df_lda_kickstarter.drop(df_lda_kickstarter.columns.difference(['project_description', 'project_category_id', 'project_parent_category_id', 'project_state']), axis=1,inplace=True)

### Technology Category

In [44]:
df_lda_kickstarter_technology = df_lda_kickstarter.copy()

In [45]:
# Filter Dataframe by Technology category (number 16)
df_lda_kickstarter_technology = df_lda_kickstarter_technology[(df_lda_kickstarter_technology['project_category_id'] == 16) | (df_lda_kickstarter_technology['project_parent_category_id'] == 16)]

In [ ]:
# Preprocessing general steps
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
df_lda_kickstarter_technology['processed_description'] = df_lda_kickstarter_technology['project_description'].progress_apply(preprocess_text_helper)

In [ ]:
data_technology = df_lda_kickstarter_technology['processed_description'].tolist()
# Compute n-grams
from gensim.models import Phrases

# Add n-grams to docs (only ones that appear 20 times or more).
ngrams_technology = Phrases(data_technology, min_count=20)

for idx in range(len(data_technology)):
    for token in ngrams_technology[data_technology[idx]]:
        if '_' in token:
            # Token is a n-grams, add to document.
            data_technology[idx].append(token)

In [50]:
# create dictionary
dictionary_technology = corpora.Dictionary(data_technology)

In [51]:
# Filter out words that occur less than 5 documents, or more than 50% of the documents.
dictionary_technology.filter_extremes(no_below=5, no_above=0.5)

In [52]:
# Define your custom stoplist
custom_stoplist_technology = ["project", "kickstarter", "pledge", "backer", "campaign", "goal", "product", "kickstarter_campaign", "funding_goal", "reward", "stretch", "stretch_goal", "fund", "funding", "pledge_level",
                            "tier", "reward_tier", "pledge_amount",
                            "play", "replay", "browser", "html5", "play_replay", "html5_browser", 
                            "people", "world", "thing", "lot", "device", "system", "user", "way", "technology", "use", "design", "year", "month", "day", "hour", "www", "com", "one"
                            "information", "opportunity", "fund", "funding", "technology", "solution", "developement", "tech", "experience", "level", "support", "stretch", "stretch_goal", "let", "detail",
                            "option", "please", "help", "life", "idea", "share", "everything", "thank", "quality", "version"]
# Add your custom stop words to the dictionary
stop_ids_technology = [dictionary_technology.token2id[word] for word in custom_stoplist_technology if word in dictionary_technology.token2id]
# Remove the stop words from the dictionary
dictionary_technology.filter_tokens(bad_ids=stop_ids_technology)

In [53]:
# create corpus
corpus_technology = [dictionary_technology.doc2bow(tokens) for tokens in data_technology]

In [17]:
from collections import Counter

# Count word occurrences
word_counts_technology = Counter()
for doc in corpus_technology:
    for word_id, count in doc:
        word = dictionary_technology[word_id]
        word_counts_technology[word] += count

# Get most frequent words
most_common_technology = word_counts_technology.most_common(100)
print(most_common_technology)

[('app', 39720), ('design', 31958), ('people', 31013), ('device', 30676), ('system', 29730), ('user', 28330), ('year', 27333), ('way', 26500), ('power', 24389), ('world', 24184), ('technology', 23965), ('video', 21718), ('goal', 20748), ('phone', 20672), ('play', 19988), ('part', 18539), ('life', 18421), ('feature', 18172), ('team', 18028), ('development', 17840), ('experience', 17774), ('day', 17587), ('software', 17564), ('data', 17200), ('business', 16974), ('battery', 16912), ('home', 16488), ('prototype', 15725), ('application', 15594), ('support', 15540), ('community', 15178), ('work', 15047), ('idea', 14839), ('cost', 14608), ('production', 14579), ('company', 14483), ('service', 14265), ('board', 14064), ('platform', 14049), ('level', 13721), ('thing', 13459), ('information', 13440), ('reward', 12976), ('tool', 12566), ('quality', 12455), ('game', 12197), ('water', 12163), ('order', 11971), ('friend', 11919), ('student', 11894), ('hand', 11880), ('control', 11869), ('computer',

In [18]:
print('Number of unique tokens: %d' % len(dictionary_technology))
print('Number of documents: %d' % len(corpus_technology))

Number of unique tokens: 19789
Number of documents: 26976


In [19]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback
pl = PerplexityMetric(corpus=corpus_technology, logger="visdom", title="Perplexity")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=corpus_technology, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=corpus_technology, texts=data_technology, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [pl, ch_umass, ch_cv, diff_kl, convergence_kl]

In [ ]:
lda_model_technology = gensim.models.LdaModel(corpus=corpus_technology, id2word=dictionary_technology, num_topics=100, passes=15, per_word_topics=True, chunksize=1500, iterations=150, alpha='auto', callbacks=callbacks)

In [ ]:
lda_model_technology.save('./LDA/technology/100/LDA_technology_100_final')

In [ ]:
vis = gensimvis.prepare(lda_model_technology, corpus_technology, dictionary_technology)
pyLDAvis.save_html(vis, './LDA/technology/vis/100_final.html')

In [ ]:
get_top_descriptions(lda_model_technology, df_lda_kickstarter_technology, corpus_technology, 62, 10)

### Games Category

In [22]:
df_lda_kickstarter_games = df_lda_kickstarter.copy()

In [23]:
# Filter Dataframe by Games category (number 12)
df_lda_kickstarter_games = df_lda_kickstarter_games[(df_lda_kickstarter_games['project_category_id'] == 12) | (df_lda_kickstarter_games['project_parent_category_id'] == 12)]

In [ ]:
# Preprocessing
df_lda_kickstarter_games['project_description'] = df_lda_kickstarter_games['project_description'].progress_apply(preprocess_text_helper)

In [25]:
data_games = df_lda_kickstarter_games['project_description'].tolist()
# Compute bigrams/trigrams
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
ngrams_games = Phrases(data_games, min_count=20)
for idx in range(len(data_games)):
    for token in ngrams_games[data_games[idx]]:
        if '_' in token:
            # Token is a bigram/trigram, add to document.
            data_games[idx].append(token)

In [26]:
# create dictionary
dictionary_games = corpora.Dictionary(data_games)

In [27]:
# Filter out words that occur less than 5 documents, or more than 50% of the documents.
dictionary_games.filter_extremes(no_below=5, no_above=0.5)

In [28]:
# Define your custom stoplist
custom_stoplist_games = ["project", "kickstarter", "pledge", "backer", "campaign", "goal", "product", "kickstarter_campaign", "funding_goal", "reward", "stretch", "stretch_goal", "fund", "funding", "pledge_level",
                        "tier", "reward_tier", "pledge_amount",
                        "play", "replay", "browser", "html5", "play_replay", "html5_browser",
                        "year", "month", "day", "hour",
                        "people", "thing", "lot", "let", "something", "anyone", "card", "help", "one", "thing", "character", "video", "level", "design", "use",
                        "system", "feature", "play", "style", "title", "feedback", "support", "version", "please", "www", "com", "life", "way", "world", "idea", "share", "everything", "thank", "quality", "version"]                       
# Add your custom stop words to the dictionary
stop_ids_games = [dictionary_games.token2id[word] for word in custom_stoplist_games if word in dictionary_games.token2id]
# Remove the stop words from the dictionary
dictionary_games.filter_tokens(bad_ids=stop_ids_games)

In [29]:
# create corpus
corpus_games = [dictionary_games.doc2bow(tokens) for tokens in data_games]

In [30]:
from collections import Counter

# Count word occurrences
word_counts_games = Counter()
for doc in corpus_games:
    for word_id, count in doc:
        word = dictionary_games[word_id]
        word_counts_games[word] += count

# Get most frequent words
most_common_games = word_counts_games.most_common(100)
print(most_common_games)

[('card', 187046), ('character', 65596), ('level', 62734), ('deck', 61114), ('book', 48977), ('reward', 48967), ('play', 44231), ('art', 41949), ('design', 41142), ('year', 40574), ('stretch', 39548), ('dice', 38203), ('stretch_goal', 37227), ('people', 35017), ('story', 34738), ('copy', 34333), ('board', 34033), ('rule', 33813), ('adventure', 32497), ('shipping', 32293), ('system', 32043), ('team', 31380), ('box', 30285), ('set', 30063), ('edition', 29871), ('version', 29191), ('item', 29123), ('cost', 28641), ('life', 27998), ('point', 27868), ('friend', 27775), ('order', 27523), ('thing', 27511), ('color', 26079), ('experience', 25413), ('hand', 25394), ('part', 25250), ('work', 25206), ('page', 25084), ('day', 23422), ('power', 23205), ('action', 22846), ('monster', 22786), ('video', 22296), ('print', 22199), ('ability', 21917), ('number', 21794), ('battle', 21661), ('map', 21426), ('idea', 20619), ('support', 20463), ('piece', 20376), ('everyone', 20289), ('fun', 20132), ('hero', 

In [31]:
print('Number of unique tokens: %d' % len(dictionary_games))
print('Number of documents: %d' % len(corpus_games))

Number of unique tokens: 31660
Number of documents: 39654


In [ ]:
# Initialize an empty list to store the LDA models
lda_models_games = []

In [41]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback
pl = PerplexityMetric(corpus=corpus_games, logger="visdom", title="Perplexity")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=corpus_games, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=corpus_games, texts=data_games, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [pl, ch_umass, ch_cv, diff_kl, convergence_kl]

In [32]:
lda_model_games = gensim.models.LdaModel(corpus=corpus_games, id2word=dictionary_games, num_topics=70, passes=15, per_word_topics=True, chunksize=2000, iterations=150, alpha='auto')

In [ ]:
lda_model_games.save('./LDA/games/70/LDA_games_70_final')

In [ ]:
vis = gensimvis.prepare(lda_model_games, corpus_games, dictionary_games)
pyLDAvis.save_html(vis, './LDA/games/vis/70_final.html')

### Design Category

In [33]:
df_lda_kickstarter_design = df_lda_kickstarter.copy()

In [34]:
# Filter Dataframe by Design category (number 7)
df_lda_kickstarter_design = df_lda_kickstarter_design[(df_lda_kickstarter_design['project_category_id'] == 7) | (df_lda_kickstarter_design['project_parent_category_id'] == 7)]

In [ ]:
# Preprocessing general steps
nltk.download('stopwords')
nltk.download('wordnet')
df_lda_kickstarter_design['project_description'] = df_lda_kickstarter_design['project_description'].progress_apply(preprocess_text_helper)

In [36]:
data_design = df_lda_kickstarter_design['project_description'].tolist()
# Compute n-grams
from gensim.models import Phrases

# Add n-grams to docs (only ones that appear 20 times or more).
ngrams_design = Phrases(data_design, min_count=20)
for idx in range(len(data_design)):
    for token in ngrams_design[data_design[idx]]:
        if '_' in token:
            # Token is a n-grams, add to document.
            data_design[idx].append(token)

In [37]:
# create dictionary
dictionary_design = corpora.Dictionary(data_design)

In [38]:
# Filter out words that occur less than 5 documents, or more than 50% of the documents.
dictionary_design.filter_extremes(no_below=5, no_above=0.5)

In [39]:
# Define your custom stoplist
custom_stoplist_design = ["project", "kickstarter", "pledge", "backer", "campaign", "goal", "product", "kickstarter_campaign", "funding_goal", "reward", "stretch", "stretch_goal", "fund", "funding", "pledge_level",
                        "tier", "reward_tier", "pledge_amount",
                        "play", "replay", "browser", "html5", "play_replay", "html5_browser",
                        "year", "month", "day", "hour",
                        "people", "life", "way", "world", "idea", "share", "everything", "thank", "quality", "version", "thing", "lot", "let", "something", "anyone", "card", "help", "one", 
                        "thing", "character", "video", "level", "design", "use", "system", "feature", "play", "style", "title", "feedback", "support", "version", "please", "www", "com"]
# Add your custom stop words to the dictionary
stop_ids_design = [dictionary_design.token2id[word] for word in custom_stoplist_design if word in dictionary_design.token2id]
# Remove the stop words from the dictionary
dictionary_design.filter_tokens(bad_ids=stop_ids_design)

In [40]:
# create corpus
corpus_design = [dictionary_design.doc2bow(tokens) for tokens in data_design]

In [41]:
from collections import Counter

# Count word occurrences
word_counts_design = Counter()
for doc in corpus_design:
    for word_id, count in doc:
        word = dictionary_design[word_id]
        word_counts_design[word] += count

# Get most frequent words
most_common_design = word_counts_design.most_common(100)
print(most_common_design)

[('year', 29868), ('way', 26199), ('production', 25489), ('color', 25434), ('material', 23784), ('goal', 23654), ('hand', 22765), ('people', 21379), ('day', 21053), ('reward', 20921), ('quality', 20735), ('bag', 20164), ('prototype', 19939), ('world', 19687), ('life', 19384), ('case', 17866), ('order', 17849), ('water', 17793), ('part', 17763), ('play', 17668), ('size', 17603), ('idea', 16367), ('card', 16111), ('support', 15584), ('work', 15074), ('process', 14131), ('home', 14068), ('tool', 13981), ('system', 13738), ('thing', 13736), ('cost', 13113), ('company', 12783), ('pocket', 12689), ('use', 12287), ('phone', 12187), ('experience', 11951), ('option', 11928), ('friend', 11918), ('piece', 11869), ('place', 11760), ('space', 11305), ('video', 11272), ('something', 11270), ('help', 11210), ('steel', 11046), ('business', 10898), ('market', 10878), ('team', 10829), ('level', 10716), ('device', 10470), ('wallet', 10402), ('family', 10353), ('watch', 10252), ('line', 10079), ('shipping

In [42]:
print('Number of unique tokens: %d' % len(dictionary_design))
print('Number of documents: %d' % len(corpus_design))

Number of unique tokens: 21627
Number of documents: 28290


In [ ]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback
pl = PerplexityMetric(corpus=corpus_design, logger="visdom", title="Perplexity")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=corpus_design, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=corpus_design, texts=data_design, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [pl, ch_umass, ch_cv, diff_kl, convergence_kl]

In [81]:
lda_model_design = gensim.models.LdaModel(corpus=corpus_design, id2word=dictionary_design, num_topics=100, passes=15, per_word_topics=True, chunksize=1500, iterations=150, alpha='auto')

In [ ]:
lda_model_design.save('./LDA/design/100/LDA_games_70_final')

In [ ]:
vis = gensimvis.prepare(lda_model_design, corpus_design, dictionary_design)
pyLDAvis.save_html(vis, './LDA/design/vis/100_final.html')